In [1]:
import argparse
import os
import warnings
import yaml

import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.accelerators import find_usable_cuda_devices
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from nanodet.data.collate import naive_collate
from nanodet.data.dataset import build_dataset
from nanodet.evaluator import build_evaluator
from nanodet.trainer.task import TrainingTask
from nanodet.trainer.dist_task import DistTrainingTask
from torchvision.transforms import ToTensor, ToPILImage
from nanodet.util import (
    NanoDetLightningLogger,
    cfg,
    convert_old_model,
    env_utils,
    load_config,
    load_model_weight,
    mkdir,
)

#Set logger and seed
logger = NanoDetLightningLogger('test')
pl.seed_everything(1234)

#Function to create the task configuration file required for training
def create_exp_cfg(yml_path, task):
    all_names = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
    #Load the YAML file
    with open(yml_path, 'r') as file:
        temp_cfg = yaml.safe_load(file)
    #Save dir of the model
    temp_cfg['save_dir'] = 'models/task' + str(task)
    #If base task, training and testing classes are the same
    if task == 0:
        temp_cfg['data']['train']['class_names'] = all_names[:15]
        temp_cfg['data']['val']['class_names'] = all_names[:15]
        temp_cfg['model']['arch']['head']['num_classes'] = 20 #15
        #temp_cfg['model']['arch']['aux_head']['num_classes'] = 20 #15
    #Else, training only on task specific class, and testing on all classes
    else:
        temp_cfg['data']['train']['class_names'] = [all_names[14+task]]
        temp_cfg['data']['val']['class_names'] = all_names[:15+task]
        temp_cfg['model']['arch']['head']['num_classes'] = 20#15+task
        #temp_cfg['model']['arch']['aux_head']['num_classes'] = 20#15+task
        temp_cfg['schedule']['load_model'] = 'models/task' + str(task-1) + '/model_last.ckpt'
        
    temp_cfg_name = 'cfg/task' + str(task) + '.yml'
    print(temp_cfg_name)
    #Save the new configuration file
    with open(temp_cfg_name, 'w') as file:
        yaml.safe_dump(temp_cfg, file)

Global seed set to 1234


In [ ]:
###LEARNING STREAM###
#task 0: train on first 15 classes, test on 15 classes
#task 1: train on class n°16, test on 16 classes
#task 2: train on class n°17, test on 17 classes
#task 3: train on class n°18, test on 18 classes
#task 4: train on class n°19, test on 19 classes
#task 5: train on class n°20, test on 20 classes
torch.set_printoptions(profile="full")
for task in range (1, 6):
    logger = NanoDetLightningLogger('run_logs/task'+str(task))
    logger.info("Starting task" + str(task))
    logger.info("Setting up data...")
    #Create the task configuration file based on the task number and load the configuration
    create_exp_cfg('cfg/VOC_dist.yml', task)
    load_config(cfg, 'cfg/task' + str(task) + '.yml')
    #Build datasets and dataloaders based on the task configuration file
    train_dataset = build_dataset(cfg.data.train, "train")
    #If task is not 0, create the replay dataset using the buffer
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=cfg.device.batchsize_per_gpu,
        shuffle=True,
        num_workers=cfg.device.workers_per_gpu,
        pin_memory=True,
        collate_fn=naive_collate,
        drop_last=True,
    )
    val_dataset = build_dataset(cfg.data.val, "test")
    val_dataloader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=cfg.device.batchsize_per_gpu,
        shuffle=False,
        num_workers=cfg.device.workers_per_gpu,
        pin_memory=True,
        collate_fn=naive_collate,
        drop_last=False,
    )
    evaluator = build_evaluator(cfg.evaluator, val_dataset)
    
    #Create the model based on the task configuration file
    logger.info("Creating models")
    if task == 0:
        TrainTask = TrainingTask(cfg, evaluator)
    else:
        TrainTask = DistTrainingTask(cfg, evaluator)
        #Load the model weights if task is not 0
        if "load_model" in cfg.schedule:
            ckpt = torch.load(cfg.schedule.load_model)
            load_model_weight(TrainTask.model, ckpt, logger)
            load_model_weight(TrainTask.teacher, ckpt, logger)
            logger.info("Loaded model weight from {}".format(cfg.schedule.load_model))
    
    model_resume_path = (
        os.path.join(cfg.save_dir, "model_last.ckpt")
        if "resume" in cfg.schedule
        else None
    )
    #Set the device to GPU if available
    if cfg.device.gpu_ids == -1:
        logger.info("Using CPU training")
        accelerator, devices, strategy, precision = (
            "cpu",
            None,
            None,
            cfg.device.precision,
        )
    else:
        accelerator, devices, strategy, precision = (
            "gpu",
            cfg.device.gpu_ids,
            None,
            cfg.device.precision,
        )

    if devices and len(devices) > 1:
        strategy = "ddp"
        env_utils.set_multi_processing(distributed=True)

    trainer = pl.Trainer(
        default_root_dir=cfg.save_dir,
        max_epochs=100,
        check_val_every_n_epoch=10,
        accelerator=accelerator,
        devices=[2],
        log_every_n_steps=cfg.log.interval,
        num_sanity_val_steps=0,
        callbacks=[TQDMProgressBar(refresh_rate=0)],
        logger=logger,
        benchmark=cfg.get("cudnn_benchmark", True),
        gradient_clip_val=cfg.get("grad_clip", 0.0),
        strategy=strategy,
        precision=precision,
    )
    trainer.fit(TrainTask, train_dataloader, val_dataloader, ckpt_path=model_resume_path)

[NanoDet][02-15 14:57:57]INFO:Starting task1
[NanoDet][02-15 14:57:57]INFO:Starting task1
[NanoDet][02-15 14:57:57]INFO:Setting up data...
[NanoDet][02-15 14:57:57]INFO:Setting up data...


cfg/task1.yml


Load 273 xml files and 625 boxes
creating index...
index created!


[NanoDet][02-15 14:57:57]INFO:Creating models
[NanoDet][02-15 14:57:57]INFO:Creating models
INFO:NanoDet:Creating models


Load 4530 xml files and 13606 boxes
creating index...
index created!
model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet-Plus Head.
model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet-Plus Head.


[NanoDet][02-15 14:57:59]INFO:Loaded model weight from models/task0/model_last.ckpt
[NanoDet][02-15 14:57:59]INFO:Loaded model weight from models/task0/model_last.ckpt
INFO:NanoDet:Loaded model weight from models/task0/model_last.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name    | Type        | Params
----------------------------------------
0 | model   | NanoDetPlus | 1.2 M 
1 | teacher | NanoDetPlus | 1.2 M 
----------------------------------------
1.2 M     Trainable params
1.2 M     Non-trainable params
2.3 M     Total params
9.397     Total estimated model params size (MB)
/home/pasti/anaconda3/envs/nanodet/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value 

Loading and preparing results...
DONE (t=1.58s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=12.24s).
Accumulating evaluation results...


[NanoDet][02-15 14:59:13]INFO:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.272
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.454
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.276
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.013
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.371
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.292
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.414
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.435
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.045
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.284
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.552

[NanoDet

DONE (t=4.69s).


[NanoDet][02-15 14:59:14]INFO:Saving model to models/task1/model_best/nanodet_model_best.pth
[NanoDet][02-15 14:59:14]INFO:Saving model to models/task1/model_best/nanodet_model_best.pth
INFO:NanoDet:Saving model to models/task1/model_best/nanodet_model_best.pth
[NanoDet][02-15 14:59:14]INFO:Val_metrics: {'mAP': 0.27196463236014523, 'AP_50': 0.45369945862626365, 'AP_75': 0.27608571823014794, 'AP_small': 0.012907943930001112, 'AP_m': 0.10418142936469553, 'AP_l': 0.37149904281698015}
[NanoDet][02-15 14:59:14]INFO:Val_metrics: {'mAP': 0.27196463236014523, 'AP_50': 0.45369945862626365, 'AP_75': 0.27608571823014794, 'AP_small': 0.012907943930001112, 'AP_m': 0.10418142936469553, 'AP_l': 0.37149904281698015}
INFO:NanoDet:Val_metrics: {'mAP': 0.27196463236014523, 'AP_50': 0.45369945862626365, 'AP_75': 0.27608571823014794, 'AP_small': 0.012907943930001112, 'AP_m': 0.10418142936469553, 'AP_l': 0.37149904281698015}
[NanoDet][02-15 14:59:16]INFO:Train|Epoch11/100|Iter40(1/4)| mem:5.25G| lr:8.01e-05

In [15]:
task = 1
logger = NanoDetLightningLogger('run_logs/task'+str(task))
logger.info("Starting task" + str(task))
logger.info("Setting up data...")
#Create the task configuration file based on the task number and load the configuration
create_exp_cfg('cfg/VOC_dist.yml', task)
load_config(cfg, 'cfg/task' + str(task) + '.yml')
#Build datasets and dataloaders based on the task configuration file
train_dataset = build_dataset(cfg.data.train, "train")
#If task is not 0, create the replay dataset using the buffer
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=cfg.device.batchsize_per_gpu,
    shuffle=True,
    num_workers=cfg.device.workers_per_gpu,
    pin_memory=True,
    collate_fn=naive_collate,
    drop_last=True,
)
val_dataset = build_dataset(cfg.data.val, "test")
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=cfg.device.workers_per_gpu,
    pin_memory=True,
    collate_fn=naive_collate,
    drop_last=False,
)
evaluator = build_evaluator(cfg.evaluator, val_dataset)

#Create the model based on the task configuration file
logger.info("Creating models")
if task == 0:
    TrainTask = TrainingTask(cfg, evaluator)
else:
    TrainTask = DistTrainingTask(cfg, evaluator)
    #Load the model weights if task is not 0
    if "load_model" in cfg.schedule:
        ckpt = torch.load(cfg.schedule.load_model)
        load_model_weight(TrainTask.model, ckpt, logger)
        load_model_weight(TrainTask.teacher, ckpt, logger)
        logger.info("Loaded model weight from {}".format(cfg.schedule.load_model))

model_resume_path = (
    os.path.join(cfg.save_dir, "model_last.ckpt")
    if "resume" in cfg.schedule
    else None
)
#Set the device to GPU if available
if cfg.device.gpu_ids == -1:
    logger.info("Using CPU training")
    accelerator, devices, strategy, precision = (
        "cpu",
        None,
        None,
        cfg.device.precision,
    )
else:
    accelerator, devices, strategy, precision = (
        "gpu",
        cfg.device.gpu_ids,
        None,
        cfg.device.precision,
    )

if devices and len(devices) > 1:
    strategy = "ddp"
    env_utils.set_multi_processing(distributed=True)

[NanoDet][02-15 13:06:39]INFO:Starting task1
[NanoDet][02-15 13:06:39]INFO:Starting task1
[NanoDet][02-15 13:06:39]INFO:Starting task1
[NanoDet][02-15 13:06:39]INFO:Starting task1
INFO:NanoDet:Starting task1
[NanoDet][02-15 13:06:39]INFO:Setting up data...
[NanoDet][02-15 13:06:39]INFO:Setting up data...
[NanoDet][02-15 13:06:39]INFO:Setting up data...
[NanoDet][02-15 13:06:39]INFO:Setting up data...
INFO:NanoDet:Setting up data...


cfg/task1.yml


Load 273 xml files and 625 boxes
creating index...
index created!


[NanoDet][02-15 13:06:39]INFO:Creating models
[NanoDet][02-15 13:06:39]INFO:Creating models
[NanoDet][02-15 13:06:39]INFO:Creating models
[NanoDet][02-15 13:06:39]INFO:Creating models
INFO:NanoDet:Creating models


Load 4530 xml files and 13606 boxes
creating index...
index created!
model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet-Plus Head.
model size is  1.0x
init weights...


[NanoDet][02-15 13:06:39]INFO:Loaded model weight from models/task0/model_last.ckpt
[NanoDet][02-15 13:06:39]INFO:Loaded model weight from models/task0/model_last.ckpt
[NanoDet][02-15 13:06:39]INFO:Loaded model weight from models/task0/model_last.ckpt
[NanoDet][02-15 13:06:39]INFO:Loaded model weight from models/task0/model_last.ckpt
INFO:NanoDet:Loaded model weight from models/task0/model_last.ckpt


=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet-Plus Head.


In [16]:
for batch in val_dataloader:
    print(batch)
    break

{'img': [tensor([[[-1.3164, -1.1247, -1.5430, -0.8110, -1.0027, -0.8458, -1.1944,
          -1.1421, -1.4210, -1.1247, -1.3687, -1.3513, -0.7587, -1.1596,
          -1.4559, -1.3513, -1.5604, -1.6127, -1.3861, -1.3861, -1.0898,
          -0.9330, -0.6193, -0.6715, -0.8110, -0.6367, -0.4624, -1.5081,
          -1.6650, -0.6018, -0.8284, -0.7761, -1.4036, -0.3230, -1.6999,
          -1.0376, -0.9156, -0.9678, -1.3339, -0.5670, -0.8807, -0.9330,
          -0.3578, -0.8633, -1.4036, -1.0898, -1.0027, -1.0898, -0.3230,
          -0.6890, -1.1073, -0.8284, -0.8110, -0.2707, -0.8110, -0.3753,
          -0.9504, -1.0201, -0.7238, -0.9678, -1.0201, -0.9330, -0.0441,
          -0.9156, -0.3753, -0.3055, -0.8458, -0.6193, -0.8284, -0.9156,
          -1.4036, -0.6890, -0.5844, -1.4384, -1.0201, -0.8981, -0.7238,
          -0.7413, -1.1421, -0.8633, -0.4798, -0.2010, -1.1247,  0.4265,
          -1.5430,  1.4548, -0.8633,  0.5834,  0.0256, -1.6999, -0.9504,
          -1.0027, -1.1073, -0.9330, -1.31

In [17]:
batch = TrainTask._preprocess_batch_input(batch)
img = batch["img"]

In [18]:
stud_feat = TrainTask.model.backbone(img)
stud_fpn_feat = TrainTask.model.fpn(stud_feat)

In [27]:
import torch
import torch.nn.functional as F
outputs = []
for feat, cls_convs in zip(
    stud_fpn_feat,
    TrainTask.model.head.cls_convs,
):
    for conv in cls_convs:
        feat = conv(feat)
    outputs.append(feat)
    

mse_list = []
for tensor1, tensor2 in zip(outputs,outputs):
    mse = F.mse_loss(tensor1, tensor2)
    mse_list.append(mse.item())   
print(mse_list)
'''
    output = gfl_cls(feat)
    outputs.append(output.flatten(start_dim=2))
outputs = torch.cat(outputs, dim=2).permute(0, 2, 1)
'''

[0.0, 0.0, 0.0, 0.0]


'\n    output = gfl_cls(feat)\n    outputs.append(output.flatten(start_dim=2))\noutputs = torch.cat(outputs, dim=2).permute(0, 2, 1)\n'

In [10]:
teacher_state_dict = TrainTask.teacher.head.state_dict()
student_state_dict = TrainTask.model.head.state_dict()
print(TrainTask.teacher.head.cls_convs[0][1])

DepthwiseConvModule(
  (depthwise): Conv2d(96, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=96, bias=False)
  (pointwise): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (dwnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pwnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): LeakyReLU(negative_slope=0.1, inplace=True)
)


In [11]:
head = TrainTask.teacher.head

In [12]:
for conv in head.cls_convs:
    print(conv)

ModuleList(
  (0): DepthwiseConvModule(
    (depthwise): Conv2d(96, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=96, bias=False)
    (pointwise): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (dwnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pwnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (1): DepthwiseConvModule(
    (depthwise): Conv2d(96, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=96, bias=False)
    (pointwise): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (dwnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pwnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): LeakyReLU(negative_slope=0.1, inplace=True)
  )
)
ModuleList(
  (0): DepthwiseConvModule(
    (depthwise): Conv2d(96

In [3]:
##################
# COPY HEAD TEST #
##################
# Get the state dict of both models
teacher_state_dict = TrainTask.teacher.head.gfl_cls.state_dict()
student_state_dict = TrainTask.model.head.gfl_cls.state_dict()
# Update the specific layer weights in the student model with the teacher model weights
for name, param in teacher_state_dict.items():
    # Slice the weights tensor along the out_channels dimension from 0 to 15
    student_state_dict[name][:17] = param[:17]
# Load the updated state dict into the student model
TrainTask.model.head.gfl_cls.load_state_dict(student_state_dict)
trainer.save_checkpoint("models/task1/model_last.ckpt")